# **MEMBACA FILE EDF**

In [ ]:
#Install pyEDFlib
!pip install pyEDFlib
!pip install mne

In [ ]:
# Import library yang diperlukan
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
import mne
from scipy.fft import fft, ifft

In [ ]:
import os

file_path = "PRIVATE_DATA/subject01.edf"

if not os.path.exists(file_path):
    # Ini yang akan muncul jika file tidak ada (Tanpa Error Merah)
    print("File EDF tidak disertakan karena bersifat privat.")
    print("Silakan gunakan data EEG Anda sendiri.")
else:
    # Kode ini HANYA jalan kalau filenya beneran ada
    print("File ditemukan")
    # Contoh: data = read_edf(file_path)
    # print(data.summary())

In [ ]:
# baca Baca file EDF
try:
    raw = mne.io.read_raw_edf(file_path, preload=True)
    print("File berhasil dibaca")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

#  **INFORMASI FILE**

In [ ]:
# Plot sinyal EEG
n_channels = len(raw.ch_names)

# Plot the raw data with all channels
raw.plot(n_channels=n_channels, duration=10, show=True)

In [ ]:
info = raw.info
data, times = raw[:]

#informasi dasar
print(f"Info: {info}")
print(f"Data Shape: {data.shape}")
print(f"Times Shape: {times.shape}")

In [ ]:
#jumlah sampel
n_samples = raw.n_times

#frekuensi sampel
sfreq = raw.info['sfreq']

# Hitung durasi dalam detik
duration = n_samples / sfreq
print(f"Durasi Rekaman: {duration} detik")

In [ ]:
#frekuensi sampel
sfreq = raw.info['sfreq']
print(f"Frekuensi Sampel: {sfreq} Hz")

In [ ]:
data.shape

# **DATA 1**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:27:32.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_1 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_1)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:27:32.100', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameter sampling
fs = raw.info['sfreq']  # Frekuensi sampling
Ts = 1 / fs             # Interval waktu sampling

# Waktu start dan durasi dalam detik
start_time_sec = start_time_total_sec  # Waktu mulai dalam detik
duration_sec = 0.5                     # Durasi dalam detik

# Hitung indeks waktu diskrit
n_values = np.arange(0, int(duration_sec * fs))  # Urutan indeks n dalam durasi
t_values = n_values * Ts + start_time_sec        # Konversi n menjadi waktu aktual

# Ekstraksi data diskrit dari sinyal raw pada waktu t_values
data, times = raw.get_data(picks=picks, start=int(start_time_sec * fs), stop=int((start_time_sec + duration_sec) * fs), return_times=True)

# Plot hasil
plt.plot(times, data.T)
plt.xlabel("Waktu (detik)")
plt.ylabel("Amplitudo (diskritisasi)")
plt.title("Diskritisasi Sinyal EEG Menggunakan Rumus x[n] = x(nT_s)")
plt.show()


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_1, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_1):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 2**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:28:57.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_2 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_2)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:56.900', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_2, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_2):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_2):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_2):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 3**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:50.600

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_3 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_3)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:50.350', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_3, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_3):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_3):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_3):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 4**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:47.200

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_4 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_4)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:47.180', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_4, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_4):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_4):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_4):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 5**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:11:48.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_5 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_5)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:11:48.530', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_5, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_5):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_5):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_5):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 6**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4


WAKTU KEJADIAN:
08:22:10.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_6 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_6)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:22:10.000', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_6, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_6):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_6):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_6):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 7**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:22:30.100

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_7 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_7)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:22:30.150', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_7, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_7):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_7):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_7):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 8**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:22:33.100

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_8 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_8)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:22:33.150', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_8, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_8):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_8):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_8):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 9**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:58.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_9 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_9)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:58.350', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_9, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_9):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_9):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_9):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 10**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:05.400

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_10 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_10)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:05.380', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_10, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_10):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_10):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_10):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 11**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:28:13.100

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_11 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_11)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:12.880', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_11, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_11):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_11):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_11):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 12**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:28:21.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_12 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_12)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:21.520', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_12, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_12):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_12):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_12):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 13**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:28:23.900

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_13 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_13)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:23.880', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_13, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_13):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_13):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_13):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 14**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:28:34.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_14 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_14)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:34.050', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_14, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_14):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_14):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_14):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 15**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:11.250

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_15 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_15)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:11.150', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_15, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_15):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_15):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_15):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 16**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:12.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_16 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_16)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:12.550', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')

In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_16, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_16):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_16):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_16):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 17**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:13.700

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_17 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_17)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:13.700', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_17, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_17):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_17):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_17):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 18**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:35.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_18 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_18)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:35.310', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_18, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_18):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_18):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_18):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 19**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:55.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_19 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_19)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:55.180', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_19, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_19):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_19):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_19):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 20**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:53.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_20 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_20)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:53.300', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_20, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_20):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_20):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_20):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 21**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:29:58.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_21 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_21)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:57.800', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_21, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_21):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_21):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_21):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print("Matriks FFT (Magnitude) 5x180:")
print(result_matrix)

# **DATA 22**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:05.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_22 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_22)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:05.375', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_22, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_22):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_22):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_22):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 23**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:24.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_23 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_23)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:24.480', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_23, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_23):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_23):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_23):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 24**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:33.100

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_24 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_24)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:32.900', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_24, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_24):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_24):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_24):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 25**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:34.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_25 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_25)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:34.000', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_25, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_25):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_25):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_25):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 26**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:45.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_26 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_26)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:45.250', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_26, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_26):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_26):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_26):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 27**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:30:58.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_27 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_27)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:58.330', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_27, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_27):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_27):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_27):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 28**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:02.800

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_28 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_28)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:02.750', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_28, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_28):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_28):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_28):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 29**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:09.200

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_29 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_29)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:09.150', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_29, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_29):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_29):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_29):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 30**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:12.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_30 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_30)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:12.520', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_30, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_30):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_30):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_30):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 31**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:17.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_31 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_31)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:16.800', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_31, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_31):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_31):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_31):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 32**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:25.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_32 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_32)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:25.365', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_32, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_32):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_32):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_32):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 33**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:28.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_33 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_33)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:28.129', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_33, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_33):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_33):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_33):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 34**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:31:46.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_34 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_34)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:46.350', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_34, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_34):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_34):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_34):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 35**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:05.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_35 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_35)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:04.820', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_35, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_35):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_35):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_35):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 36**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:13.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_36 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_36)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:13.657', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_36, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_36):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_36):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_36):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 37**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:18.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_37 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_37)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:17.852', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_37, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_37):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_37):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_37):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 38**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:27.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_38 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_38)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:26.877', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_38, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_38):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_38):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_38):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 39**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:29.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_39 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_39)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:29.677', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_39, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_39):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_39):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_39):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **DATA 40**

CHANNEL YANG INGIN DITAMPILKAN:
EEG T2-T4



WAKTU KEJADIAN:
08:32:44.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_40 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_40)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:44.596', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_40, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_40):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_40):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_data_40):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 1**

WAKTU KEJADIAN:
08:26:59.300

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_1 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_1)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:26:59.300', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_1, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_1):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 2**

WAKTU KEJADIAN:
08:28:16.900

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_2 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_2)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:16.900', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_2, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_2):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_2):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_2):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 3**

WAKTU KEJADIAN:
08:30:52.700

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_3 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_3)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:52.700', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_3, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_3):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_3):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_3):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 4**

WAKTU KEJADIAN:
08:32:00.900

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_4 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_4)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:32:00.900', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_4, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_4):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_4):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_4):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 5**

WAKTU KEJADIAN:
08:31:42.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_5 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_5)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:31:42.100', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_5, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_5):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_5):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_5):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 6**

WAKTU KEJADIAN:
08:28:17.900

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_6 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_6)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:16.900', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_6, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_6):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_6):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_6):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 7**

WAKTU KEJADIAN:
08:28:20.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_7 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_7)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:20.050', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_7, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_7):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_7):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_7):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 8**

WAKTU KEJADIAN:
08:28:22.800

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_8 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_8)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:22.820', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_8, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_8):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_8):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_8):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 9**

WAKTU KEJADIAN:
08:28:27.700

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_9 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_9)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:27.680', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_9, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_9):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_9):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_9):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 10**

WAKTU KEJADIAN:
08:28:32.600

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_10 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_10)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:32.650', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_10, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_10):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_10):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_10):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 11**

WAKTU KEJADIAN:
08:28:36.600

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_11 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_11)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:36.580', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_11, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_11):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_11):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_11):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 12**

WAKTU KEJADIAN:
08:28:50.100

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_12 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_12)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:50.060', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_12, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_12):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_12):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_12):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 13**

WAKTU KEJADIAN:
08:30:17.500

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_13 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_13)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:17.470', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_13, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_13):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_13):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_13):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 14**

WAKTU KEJADIAN:
08:30:42.400

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_14 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_14)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:42.400', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_14, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_14):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_14):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_14):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 15**

WAKTU KEJADIAN:
08:28:26.950

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_15 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_15)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:26.691', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_15, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_15):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_15):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_15):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 16**

WAKTU KEJADIAN:
08:28:30.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_16 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_16)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:30.201', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_16, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_16):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_16):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_16):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 17**

WAKTU KEJADIAN:
08:28:39.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_17 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_17)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:39.002', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_17, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_17):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_17):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_17):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 18**

WAKTU KEJADIAN:
08:28:43.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_18 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_18)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:43.233', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_18, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_18):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_18):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_18):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 19**

WAKTU KEJADIAN:
08:28:46.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_19 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_19)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:45.907', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_19, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_19):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_19):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_19):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 20**

WAKTU KEJADIAN:
08:28:58.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_20 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_20)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:28:58.242', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_20, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_20):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_20):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_20):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 21**

WAKTU KEJADIAN:
08:29:05.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_21 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_21)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:05.156', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_21, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_21):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_21):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_21):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print("Matriks FFT (Magnitude) 5x180:")
print(result_matrix)

# **NORMAL 22**

WAKTU KEJADIAN:
08:29:06.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_22 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_22)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:06.596', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_22, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_22):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_22):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_22):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 23**

WAKTU KEJADIAN:
08:29:09.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_23 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_23)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:08.841', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_23, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_23):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_23):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_23):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 24**

WAKTU KEJADIAN:
08:29:16.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_24 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_24)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:16.355', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_24, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_24):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_24):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_24):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 25**

WAKTU KEJADIAN:
08:29:19.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_25 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_25)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:18.940', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_25, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_25):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_25):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_25):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 26**

WAKTU KEJADIAN:
08:29:20.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_26 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_26)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:20.025', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_26, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_26):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_26):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_26):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 27**

WAKTU KEJADIAN:
08:29:24.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_27 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_27)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:24.373', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_27, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_27):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_27):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_27):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 28**

WAKTU KEJADIAN:
08:29:30.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_28 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_28)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:30.428', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_28, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_28):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_28):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_28):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 29**

WAKTU KEJADIAN:
08:29:42.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_29 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_29)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:42.637', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_29, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_29):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_29):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_29):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 30**

WAKTU KEJADIAN:
08:29:40.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_30 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_30)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:40.607', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_30, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_30):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_30):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_30):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 31**

WAKTU KEJADIAN:
08:29:45.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_31 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_31)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:45.634', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_31, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_31):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_31):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_31):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print("Matriks FFT (Magnitude) 5x180:")
print(result_matrix)

# **NORMAL 32**

WAKTU KEJADIAN:
08:29:46.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_32 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_32)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:46.305', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_32, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_32):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_32):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_32):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 33**

WAKTU KEJADIAN:
08:29:48.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_33 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_33)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:47.852', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_33, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_33):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_33):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_33):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 34**

WAKTU KEJADIAN:
08:29:50.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_34 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_34)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:49.802', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_34, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_34):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_34):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_34):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 35**

WAKTU KEJADIAN:
08:29:51.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_35 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_35)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:29:50.839', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_35, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_35):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_35):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_35):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 36**

WAKTU KEJADIAN:
08:30:01.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_36 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_36)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:01.054', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_36, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_36):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_36):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_36):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 37**

WAKTU KEJADIAN:
08:30:03.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_37 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_37)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:03.585', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_37, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_37):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_37):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_37):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 38**

WAKTU KEJADIAN:
08:30:10.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_38 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_38)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:10.473', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_38, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_38):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_38):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_38):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 39**

WAKTU KEJADIAN:
08:30:12.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_39 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_39)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:12.530', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_39, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_39):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_39):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_39):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **NORMAL 40**

WAKTU KEJADIAN:
08:30:40.000

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_normal_40 = ['EEG Fp2-F8', 'EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6', 'EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_normal_40)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:30:40.446', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_sec, duration_sec, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_sec * sfreq)
    end_sample = int((start_time_sec + duration_sec) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_sec - 0.5, duration_sec, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_sec, duration_sec, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_sec + 0.5, duration_sec, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_normal_40, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_normal_40):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_normal_40):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

# Proses setiap channel
for i, channel_name in enumerate(channel_normal_40):
    # Menghitung FFT untuk tiap interval
    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    # Menyusun hasil dalam bentuk matriks
    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

    # Menampilkan matriks hasil
print(result_matrix)

# **CATATAN/TESTING**

In [ ]:
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menghitung FFT dan mengembalikan magnitudo dalam rentang frekuensi 1-60 Hz
def calculate_fft_magnitude(channel_data, sfreq):
    N = len(channel_data)  # Panjang data
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 1 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    # Hitung FFT dan magnitudo
    X = fft(channel_data)
    X = np.abs(X[positive_freq_ indices])

    # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
    full_freq_range = np.arange(1, 61)
    X_interp = np.interp(full_freq_range, freq, X)

    return X_interp

result_matrix = np.zeros((5, 180))

for i, channel_name in enumerate(channel_data_1):

    data_before = calculate_fft_magnitude(data_sebelum[i], sfreq)
    data_now = calculate_fft_magnitude(data_saat_ini[i], sfreq)
    data_after = calculate_fft_magnitude(data_setelah[i], sfreq)

    result_matrix[i, :60] = data_before
    result_matrix[i, 60:120] = data_now
    result_matrix[i, 120:180] = data_after

print("Matriks FFT (Magnitude) 3x180:")
print(result_matrix)

In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_sec = (datetime.strptime('08:10:27.300', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:26.000', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_sec = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_sec = min(0.5, recording_duration_sec - start_time_total_sec)

# Plot data
fig = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks))

# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_sec - duration_sec)
fig_before = raw.plot(start=start_before, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_sec, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_sec + duration_sec
fig_after = raw.plot(start=start_after, duration=duration_sec, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
from datetime import datetime

# Waktu rekaman
start_time_total_1 = (datetime.strptime('08:10:27.300', '%H:%M:%S.%f') -
                        datetime.strptime('08:10:25.991', '%H:%M:%S.%f')).total_seconds()

# Durasi total rekaman
recording_duration_1 = raw.times[-1]  # Durasi rekaman dalam detik

# Durasi yang diinginkan
duration_1 = min(0.5, recording_duration_1 - start_time_total_1)

# Plot data
fig = raw.plot(start=start_time_total_1, duration=duration_1, picks=picks, n_channels=len(picks))

CHANNEL YANG INGIN DITAMPILKAN:
1. EEG F8-T2

WAKTU KEJADIAN:
08:10:27

In [ ]:
# Nama-nama channel yang ingin ditampilkan
channel_data_1 = ['EEG Fp2-F8','EEG F8-T2', 'EEG T2-T4', 'EEG T4-T6','EEG T6-O2']

# indeks channel
picks = mne.pick_channels(raw.info['ch_names'], include=channel_data_1)

In [ ]:
# Waktu rekaman yang diinginkan
start_time_1 = '08:10:27.300'  # Waktu mulai rekaman yang diinginkan
end_time_1 = '08:10:27.800'    # Waktu akhir rekaman yang diinginkan

# Konversi waktu rekaman ke dalam detik relatif terhadap waktu mulai rekaman
start_time = (datetime.strptime(start_time_1, '%H:%M:%S.%f') -
              datetime.strptime(raw.info['meas_date'].strftime('%H:%M:%S.%f'), '%H:%M:%S.%f')).total_seconds()

end_time = (datetime.strptime(end_time_1, '%H:%M:%S.%f') -
            datetime.strptime(raw.info['meas_date'].strftime('%H:%M:%S.%f'), '%H:%M:%S.%f')).total_seconds()

# Durasi yang diinginkan
duration = end_time - start_time

# Plot data
fig = raw.plot(start=start_time, duration=duration, picks=picks, n_channels=len(picks))



In [ ]:
# Plot 0.5 detik sebelum waktu yang ditentukan
start_before = max(0, start_time_total_1 - duration_1)
fig_before = raw.plot(start=start_before, duration=duration_1, picks=picks, n_channels=len(picks), title='0.5 Detik Sebelum')

# Plot 0.5 detik pada waktu yang ditentukan
fig_now = raw.plot(start=start_time_total_1, duration=duration_1, picks=picks, n_channels=len(picks), title='0.5 Detik Sekarang')

# Plot 0.5 detik setelah waktu yang ditentukan
start_after = start_time_total_1 + duration_1
fig_after = raw.plot(start=start_after, duration=duration_1, picks=picks, n_channels=len(picks), title='0.5 Detik Setelah')


In [ ]:
# Fungsi untuk ekstraksi data
def ekstrak_data(raw, start_time_1, duration_1, picks):
    sfreq = raw.info['sfreq']
    start_sample = int(start_time_1 * sfreq)
    end_sample = int((start_time_1 + duration_1) * sfreq)
    data, times = raw[picks, start_sample:end_sample]
    return data, times

# Ekstrak data dari tiga interval
data_sebelum, times_sebelum = ekstrak_data(raw, start_time_total_1 - 0.5, duration_1, picks)
data_saat_ini, times_saat_ini = ekstrak_data(raw, start_time_total_1, duration_1, picks)
data_setelah, times_setelah = ekstrak_data(raw, start_time_total_1 + 0.5, duration_1, picks)

# Buat objek Info baru
info = mne.create_info(ch_names=channel_data_1, sfreq=sfreq, ch_types='eeg')

# Buat dan plot objek RawArray untuk setiap interval
intervals = [
    (data_sebelum, '0.5 Detik Sebelum'),
    (data_saat_ini, 'Saat Ini'),
    (data_setelah, '0.5 Detik Setelah')
]

In [ ]:
def plot_channel_fft_ifft(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 Detik Sebelum'),
        (channel_data[1], 'Saat Ini'),
        (channel_data[2], '0.5 Detik Setelah')
    ]

    N = len(channel_data[0])
    T = 1.0 / sfreq
    x = np.linspace(0.0, N*T, N, endpoint=False)
    freq = np.fft.fftfreq(N, T)[:N//2]

    fig, axs = plt.subplots(3, 3, figsize=(16, 8))

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)

        # Plot IFFT
        axs[0, j].plot(x, ifft(X), 'r')
        axs[0, j].set_title(f'IFFT {interval}')
        axs[0, j].set_xlabel('Waktu (detik)')
        axs[0, j].set_ylabel('Amplitudo')

        # Plot FFT dengan amplitudo
        axs[1, j].stem(freq, 2.0/N * np.abs(X[:N//2]), 'b', markerfmt=" ", basefmt="-b")
        axs[1, j].set_title(f'Amplitudo FFT {interval}')
        axs[1, j].set_xlabel('Frekuensi (Hz)')
        axs[1, j].set_ylabel('Amplitudo (2/N * |X(freq)|)')
        axs[1, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

        # Plot FFT dengan magnitudo
        axs[2, j].stem(freq, np.abs(X[:N//2]), 'g', markerfmt=" ", basefmt="-g")
        axs[2, j].set_title(f'Magnitudo FFT {interval}')
        axs[2, j].set_xlabel('Frekuensi (Hz)')
        axs[2, j].set_ylabel('Magnitudo |X(freq)|')
        axs[2, j].set_xlim(0, 60)  # Batasi tampilan frekuensi hingga 60 Hz

    plt.suptitle(f'Channel {channel_name}')
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Plot FFT dan IFFT untuk setiap channel
for i, channel_name in enumerate(channel_data_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    plot_channel_fft_ifft(channel_data, sfreq, channel_name)


In [ ]:
from tabulate import tabulate
import numpy as np
from scipy.fftpack import fft

# Fungsi untuk menampilkan nilai numerik FFT dari frekuensi 0 hingga 60 Hz
def display_numerical_values(channel_data, sfreq, channel_name):
    intervals = [
        (channel_data[0], '0.5 detik sebelum'),
        (channel_data[1], 'Saat ini'),
        (channel_data[2], '0.5 detik setelah')
    ]

    N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
    T = 1.0 / sfreq
    freq = np.fft.fftfreq(N, T)

    # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
    positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
    freq = freq[positive_freq_indices]

    for j, (data, interval) in enumerate(intervals):
        X = fft(data)
        X = np.abs(X[positive_freq_indices])

        # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
        full_freq_range = np.arange(1, 61)
        X_interp = np.interp(full_freq_range, freq, X)

        print(f'\nChannel {channel_name} - {interval}')

        # Tabel Magnitudo dari 1 hingga 60 Hz
        print(f'FFT Values (Magnitude) from 1 to 60 Hz - {interval}:')
        print(tabulate(np.column_stack((full_freq_range, X_interp)), headers=['Frequency (Hz)', 'Magnitude'], tablefmt='grid'))

# Menampilkan nilai numerik untuk setiap channel
for i, channel_name in enumerate(channel_data_1):
    channel_data = [
        data_sebelum[i],
        data_saat_ini[i],
        data_setelah[i]
    ]
    display_numerical_values(channel_data, sfreq, channel_name)

In [ ]:
# prompt: buatkan hasil numerik di atas dalam bentuk matrix, 5x180, baris menyatakan channel, 180 menyatakan nilai numerik setiap frekuensi

import numpy as np
def create_fft_matrix(channel_data_list, sfreq):
    num_channels = len(channel_data_list)
    fft_matrix = np.zeros((num_channels, 180))  # Inisialisasi matriks 5x180

    for i, channel_data in enumerate(channel_data_list):
        N = len(channel_data[0])  # Asumsi semua interval memiliki panjang yang sama
        T = 1.0 / sfreq
        freq = np.fft.fftfreq(N, T)

        # Ambil frekuensi dari 0 hingga 60 Hz saja, dan abaikan frekuensi negatif
        positive_freq_indices = np.where((freq >= 1) & (freq <= 60))[0]
        freq = freq[positive_freq_indices]

        for j in range(3):  # Untuk setiap interval (sebelum, saat ini, setelah)
            X = fft(channel_data[j])
            X = np.abs(X[positive_freq_indices])

            # Interpolasi untuk memastikan semua frekuensi dari 1 hingga 60 Hz
            full_freq_range = np.arange(1, 61)
            X_interp = np.interp(full_freq_range, freq, X)

            # Isi matriks FFT
            start_index = j * 60
            end_index = (j + 1) * 60
            fft_matrix[i, start_index:end_index] = X_interp

    return fft_matrix

# Buat daftar data channel
channel_data_list = [
    [data_sebelum[i], data_saat_ini[i], data_setelah[i]]
    for i in range(len(channel_data_1))
]

# Buat matriks FFT
fft_matrix = create_fft_matrix(channel_data_list, sfreq)

print(fft_matrix)
print(fft_matrix.shape)  # Verifikasi bentuk matriks (5, 180)
